In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib notebook

## DATA

In [64]:
df = pd.read_excel('DATA GGC.xlsx', sheet_name='RAPI', usecols='A:M', index_col=0)
df = df.rename(columns={df.columns[3]:'T (c)', df.columns[4]:'Ph', df.columns[5]:'H2O (%)',
                        df.columns[8]:'Ph Normal', df.columns[9]:'H2O (%) Normal'})
df

,Kode,X,Y,T (c),Ph,H2O (%),CO2 (%),Hg (ppb),Ph Normal,H2O (%) Normal,CO2 (%) Normal,Hg (ppb) Normal
Νο.,,,,,,,,,,,,
1,B-400,485532.3,9072204.0,28.7,6.21,4.97,0.4500,456.000,38.5641,0.696356,-0.346787,2.658965
2,-850,485317.9,9072591.0,30.2,6.52,6.68,0.4200,411.000,42.5104,0.824776,-0.376751,2.613842
3,-1500,485005.0,9073171.0,31.7,6.93,5.12,0.5300,544.000,48.0249,0.709270,-0.275724,2.735599
4,-2000,484737.0,9073593.0,28.9,6.06,6.09,0.5500,613.000,36.7236,0.784617,-0.259637,2.787460
5,-2500,484482.2,9074025.0,27.7,5.90,4.99,0.3300,306.000,34.8100,0.698101,-0.481486,2.485721
...,...,...,...,...,...,...,...,...,...,...,...,...
88,RanTMT,484502.0,9076138.0,25.2,2.03,7.16,69.3470,3188.280,4.1209,0.854913,1.841028,3.503556
89,TPWO,484704.0,9076826.0,44.7,3.37,9.52,16.9449,1878.869,11.3569,0.978637,1.229039,3.273897
90,CDI,484686.0,9076515.0,25.5,5.05,12.24,0.1683,173.200,25.5025,1.087781,-0.773916,2.238548


## CORRELATION 

In [65]:
from scipy import stats
p_Hg_CO2 = stats.pearsonr(np.array(df['CO2 (%)']), np.array(df['Hg (ppb)']))
print(f'Pearson Correlation Hg & CO2 : {p_Hg_CO2[0]:.3f}')

Pearson Correlation Hg & CO2 : 0.898


In [66]:
p_Hg_Ph = stats.pearsonr(np.array(df['Ph']), np.array(df['Hg (ppb)']))
print(f'Pearson Correlation Hg & Ph : {p_Hg_Ph[0]:.3f}')

Pearson Correlation Hg & Ph : -0.739


## ML

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

In [68]:
df.columns

Index(['Kode', 'X', 'Y', 'T (c)', 'Ph', 'H2O (%)', 'CO2 (%)', 'Hg (ppb)',
       'Ph Normal', 'H2O (%) Normal', 'CO2 (%) Normal', 'Hg (ppb) Normal'],
      dtype='object')

DATA SPLIT

In [69]:
X = df[['X', 'Y', 'Ph Normal', 'CO2 (%)']]
y = df['Hg (ppb) Normal']

scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#keknya data yang 'scaled' ga dipake

### KNN-Regressor 

Random Search

In [70]:
n_neighbors = [3, 4, 5, 6, 7, 8, 9, 10]
weights = ['uniform', 'distance']
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
leaf_size = [30, 40, 50, 60, 70]                         
p = [1, 2]
random_grid = {'n_neighbors': n_neighbors,
               'weights': weights,
               'algorithm': algorithm,
               'leaf_size': leaf_size,
               'p': p}

In [84]:
knr = KNeighborsRegressor()
knr_random = RandomizedSearchCV(estimator = knr, param_distributions = random_grid,
                                n_iter = 100, cv = 5, verbose=2, random_state=11,
                                n_jobs = -1)

# Fit the random search model
knr_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 308 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.9s finished
C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                                 metric='minkowski',
                                                 metric_params=None,
                                                 n_jobs=None, n_neighbors=5,
                                                 p=2, weights='uniform'),
                   iid='warn', n_iter=100, n_jobs=-1,
                   param_distributions={'algorithm': ['auto', 'ball_tree',
                                                      'kd_tree', 'brute'],
                                        'leaf_size': [30, 40, 50, 60, 70],
                                        'n_neighbors': [3, 4, 5, 6, 7, 8, 9,
                                                        10],
                                        'p': [1, 2],
                                        'weights': ['uniform', 'distance']},
                   pre_di

In [85]:
knr_random.best_score_

-0.3346836798866311

In [86]:
knr_random.best_params_

{'weights': 'uniform',
 'p': 1,
 'n_neighbors': 7,
 'leaf_size': 60,
 'algorithm': 'kd_tree'}

In [87]:
#Evaluasi RandomizedSearchCV
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print(f'MAE: {mean_absolute_error(test_labels, model.predict(test_features))}')
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy


base_model = knr
base_model.fit(X_train_scaled, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

Model Performance
MAE: 0.1424261897303464
Accuracy = 94.45%.


In [88]:
best_random_knn = knr_random.best_estimator_
random_accuracy_knn = evaluate(best_random_knn, X_test, y_test)

Model Performance
MAE: 0.19543052279711634
Accuracy = 92.29%.


GRIDSEARCH

In [89]:
grid_search_knn = GridSearchCV(estimator = knr, param_grid = random_grid, cv = 5, n_jobs = -1, verbose = 2)
grid_search_knn.fit(X_train, y_train)

Fitting 5 folds for each of 640 candidates, totalling 3200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 532 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 3200 out of 3200 | elapsed:    9.8s finished
C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='warn', n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [30, 40, 50, 60, 70],
                         'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [90]:
grid_search_knn.best_score_

-0.3346836798866311

In [91]:
grid_search_knn.best_params_

{'algorithm': 'auto',
 'leaf_size': 30,
 'n_neighbors': 7,
 'p': 1,
 'weights': 'uniform'}

In [92]:
best_grid_knn = grid_search_knn.best_estimator_
grid_accuracy = evaluate(best_grid_knn, X_test_scaled, y_test)

Model Performance
MAE: 0.15696640051839802
Accuracy = 94.14%.
